# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `三つの線分 (x, y, z) が三角形を成すか判定し、Yes/No を返す。`
* 入力 DF: `Triangle(x: int, y: int, z: int)`（各行は 1 つの三つ組）
* 出力: `x, y, z, triangle`（`triangle` は `'Yes'` または `'No'`）

## 2) 実装（指定シグネチャ厳守）

> **列最小化 → ベクトル化判定 → 必要列のみ出力**。
> オーバーフロー回避のため `int64` に昇格し、行ごとに昇順ソート（`a ≤ b ≤ c`）して **`a + b > c`** を一発判定。

```python
import pandas as pd
import numpy as np

def judge_triangle(Triangle: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        Triangle (pd.DataFrame): 入力列は x, y, z（整数想定）
    Returns:
        pd.DataFrame: 列名と順序は ['x', 'y', 'z', 'triangle']
    """
    # 必要列のみを NumPy 配列化（int64 で安全に演算）
    sides = Triangle[['x', 'y', 'z']].to_numpy(dtype='int64', copy=False)

    # 行ごとに昇順へ並べ替え: a <= b <= c
    sorted_sides = np.sort(sides, axis=1)
    a = sorted_sides[:, 0]
    b = sorted_sides[:, 1]
    c = sorted_sides[:, 2]

    # 正の長さのみ許容し、三角不等式 a + b > c で判定
    valid_positive = (a > 0) & (b > 0) & (c > 0)
    is_triangle = valid_positive & ((a + b) > c)

    # 出力 DataFrame（元の x, y, z をそのまま返し、判定を付与）
    out = Triangle[['x', 'y', 'z']].copy()
    out['triangle'] = np.where(is_triangle, 'Yes', 'No')

    return out

Analyze Complexity
Runtime 253 ms
Beats 94.30%
Memory 66.92 MB
Beats 13.23%

```

* ベクトル化のみで完結（`apply`/ループ不使用）
* `np.sort` で 3 要素の並べ替えを一括処理
* 加算は `int64`（`bigint` 相当）で安全側

## 3) アルゴリズム説明

* 使用 API / 手法

  * `DataFrame.to_numpy(dtype='int64')`: 必要列のみを配列化して高速演算
  * `np.sort(axis=1)`: 行方向に 3 要素を昇順ソート（`a ≤ b ≤ c`）
  * ベクトル化ブール式: `(a > 0) & (b > 0) & (c > 0) & ((a + b) > c)`
  * `np.where`: 真偽を `'Yes'/'No'` へマッピング
* **NULL / 重複 / 型**

  * 欠損がある場合は `to_numpy` で `float64` 化され得るため、事前に `Triangle[['x','y','z']].notna().all(1)` で除外するか、`fillna(0)` 等で方針を明示すると良い（本問題は整数前提）。
  * 重複行は仕様上そのまま返す（ID 基準）。
  * オーバーフローは `int64` 昇格で回避。

## 4) 計算量（概算）

* 行ごとの定数長（3 要素）ソート＋比較のみ：**O(N)**

  * `np.sort` は 3 要素の固定長なので実質的に定数時間/行
  * 追加メモリは 3 列分の配列程度

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 DataFrame x y z]
  B[NumPy配列化 int64]
  C[各行を昇順へ並べ替え a b c]
  D[正の長さかつ a+b>c を判定]
  E[Yes/No を付与]
  F[出力 x y z triangle]
  A --> B
  B --> C
  C --> D
  D --> E
  E --> F
```

そのスコアなら十分速い部類ですが、**ランタイム微改善＋メモリ半減**は狙えます。主なボトルネックは `np.sort(axis=1)` が **3 列×全行の並べ替え配列を丸ごと作る**点（= 追加 3 列分の一時配列）。ここを **並べ替え不要の数式**に変えると、メモリも CPU も少し軽くなります。

---

## 改善の芯

* **ソート排除**：`a ≤ b ≤ c` を作らず、
  `amin = min(x,y,z)`, `amax = max(x,y,z)`, `b = (x+y+z) - amin - amax` として **1 回の不等式 `amin + b > amax`** で判定。
* **一時配列を縮小**：並べ替えず、中間値を算術で求めて追加配列を作らない。
  ※列型は `int64` を維持しておけば境界値でも安全です。

---

## 改善版（指定シグネチャ厳守・ベクトル化のみ）

```python
import pandas as pd
import numpy as np

def judge_triangle(Triangle: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        Triangle (pd.DataFrame): 入力列は x, y, z（整数想定）
    Returns:
        pd.DataFrame: 列名と順序は ['x', 'y', 'z', 'triangle']
    """
    # 必要列のみ取り出し。int64 のまま保持して境界値でも安全に演算
    sides = Triangle[['x', 'y', 'z']].to_numpy(dtype=np.int64, copy=False)

    # 行方向の最小・最大・合計（合計のみ int64 でオーバーフロー回避）
    amin = np.minimum.reduce(sides, axis=1)
    amax = np.maximum.reduce(sides, axis=1)
    asum = sides.sum(axis=1, dtype=np.int64)                 # int64

    # 中間値 b = sum - min - max（ここからは int64 で比較）
    b = asum - amin.astype(np.int64) - amax.astype(np.int64)

    # 正の長さのみ許容
    all_pos = (sides > 0).all(axis=1)

    # 三角不等式：amin + b > amax
    is_triangle = all_pos & ((amin.astype(np.int64) + b) > amax.astype(np.int64))

    out = Triangle[['x', 'y', 'z']].copy()
    out['triangle'] = np.where(is_triangle, 'Yes', 'No')
    return out

Analyze Complexity
Runtime 255 ms
Beats 93.33%
Memory 66.76 MB
Beats 27.91%
```

### ここが効く

* **メモリ**: `np.sort` の 3 列分一時配列を作らないため、ピークメモリがぐっと下がります。
  ソートを排除しているので追加配列を確保せずに済みます。
* **CPU**: 3 要素のソート（定数時間とはいえメモリアクセス多め）を排除。
  置換後は Reduce（min/max/sum）3 回＋数回のブール演算のみ。

---

## 追加の微工夫（任意）

* 欠損が来る可能性があるなら、先に
  `mask = Triangle[['x','y','z']].notna().all(axis=1)` で絞るか、`fillna(0)` を明示。
* 大きな値域でも `int64` のままなので追加のケアは不要です。

---

## 期待効果（目安）

* **Runtime**: 数 % ～十数 % 程度の微改善（データサイズと環境依存）。
* **Memory**: ソート用の一時配列が無くなるぶんピークが下がります。

今の 253 ms / 66.9 MB は十分優秀ですが、上記に差し替えると **同等以上の速度でメモリを目に見えて削減**できるはずです。

